In [14]:
import torch
import numpy as np
import random
from IPython import display
from matplotlib import pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [15]:
num_samples=1000
num_input=2
true_w=torch.tensor([2,-3.4,2.1],dtype=torch.float64)
true_b=torch.tensor([4.2],dtype=torch.float64)
features=torch.from_numpy(np.random.normal(0,1,(num_samples,num_input)))
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b[0]
labels+=torch.from_numpy(np.random.normal(0,0.01,num_samples))

In [16]:
def data_iter(features,labels,batch_size):
    f_len=len(features)
    indices=list(range(0,f_len))
    np.random.shuffle(indices)
    for i in range(0,f_len,batch_size):
        j=torch.LongTensor(indices[i:min(i+batch_size,f_len)])
        yield features.index_select(0,j),labels.index_select(0,j)

In [17]:
for i,j  in data_iter(features,labels,50):
    print(i,j)
    break

tensor([[ 0.0628, -0.2925],
        [-0.4431,  0.1392],
        [-0.5671,  0.2262],
        [-1.2307,  0.2091],
        [-0.0492,  0.2277],
        [-0.1418, -0.3554],
        [-0.4280, -1.9641],
        [ 0.3402,  0.0057],
        [-2.4252, -0.2089],
        [ 0.3634,  0.0746],
        [ 0.4223, -1.2214],
        [ 0.6238,  1.6083],
        [-1.7483, -2.0573],
        [ 0.5167, -0.1906],
        [-2.5024, -0.6466],
        [ 1.3207,  0.5298],
        [-1.2790, -0.0135],
        [ 0.9656,  0.3033],
        [ 0.6165,  1.0595],
        [ 1.7639, -0.6480],
        [ 0.1029,  0.6386],
        [-2.2616, -1.1050],
        [ 0.2739,  0.6882],
        [ 1.4273,  0.1452],
        [-1.0170, -0.6847],
        [-0.3151, -0.4429],
        [ 2.5980, -0.7195],
        [ 0.1048,  0.0062],
        [ 0.2139, -0.6376],
        [-1.3277, -0.0759],
        [ 0.8873,  0.3572],
        [-0.9285,  1.1900],
        [-0.2044,  0.8779],
        [-0.2345,  2.0080],
        [ 0.9650,  0.4840],
        [ 1.0631, -0

In [18]:
def linreg(w,x,b):
    return torch.mm(w,x)+b

In [19]:
def l2_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2/2

In [20]:
def sgd(x,lm,params):
    for param in params:
        param.data -= lm * param.grad / len(x)

In [21]:
np.random.rand(2).tolist()

[0.35207464505345554, 0.2889197740499131]

In [22]:
w=torch.tensor(np.random.rand(2).tolist(),dtype=torch.float64).view(2,1)
b=torch.tensor([1.0],dtype=torch.float64)
batch_size=10

In [23]:
w.requires_grad_(True)
b.requires_grad_(True)
lm=1
epochs=3
for i in range(epochs):
    for x ,y in data_iter(features,labels,batch_size):
        l=l2_loss(linreg(x,w,b),y).sum()
        l.backward()
        sgd(x,lm,[w,b])
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l=l2_loss(linreg(features,w,b),labels)
    print('epoch %d ,loss %f' % (i, train_l.mean().item()))
    

epoch 0 ,loss 0.000078
epoch 1 ,loss 0.000133
epoch 2 ,loss 0.000070


In [24]:
print(true_w, '\n', w)
print(true_b, '\n', b)

tensor([ 2.0000, -3.4000,  2.1000], dtype=torch.float64) 
 tensor([[ 1.9946],
        [-3.4020]], dtype=torch.float64, requires_grad=True)
tensor([4.2000], dtype=torch.float64) 
 tensor([4.1980], dtype=torch.float64, requires_grad=True)


In [25]:
n_train,n_test,num_inputs=20,100,200
true_w,true_b=torch.ones(num_inputs,1)*0.01,0.05
features=torch.randn((n_train+n_test,num_inputs))
labels=torch.matmul(features,true_w)+true_b
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)
train_features,test_features=features[:n_train,:],features[n_train:,:]
train_labels,test_labels=labels[:n_train,:],labels[n_train:,:]

In [27]:
def init_params():
    w=torch.randn((num_inputs,1),requires_grad=True)
    b=torch.zeros(1,requires_grad=True)
    return [w,b]

In [29]:
w,b=init_params()
lm=2
epochs=100
for i in range(epochs):
    for x ,y in data_iter(train_features,train_labels,20):
        l=l2_loss(linreg(x,w,b),y).sum()
        l.backward()
        sgd(x,lm,[w,b])
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l=l2_loss(linreg(features,w,b),labels)
    print('epoch %d ,loss %f' % (i, train_l.mean().item()))
    

epoch 0 ,loss 28009.923828
epoch 1 ,loss 20538112.000000
epoch 2 ,loss 16657257472.000000
epoch 3 ,loss 14250079682560.000000
epoch 4 ,loss 12591792918626304.000000
epoch 5 ,loss 11368463147552735232.000000
epoch 6 ,loss 10423288603573233909760.000000
epoch 7 ,loss 9668270033963750470451200.000000
epoch 8 ,loss 9049403597400720230561349632.000000
epoch 9 ,loss 8531359285874947720172116377600.000000
epoch 10 ,loss 8089976532871177943254298676690944.000000
epoch 11 ,loss inf
epoch 12 ,loss inf
epoch 13 ,loss inf
epoch 14 ,loss inf
epoch 15 ,loss inf
epoch 16 ,loss inf
epoch 17 ,loss inf
epoch 18 ,loss inf
epoch 19 ,loss inf
epoch 20 ,loss inf
epoch 21 ,loss inf
epoch 22 ,loss inf
epoch 23 ,loss inf
epoch 24 ,loss inf
epoch 25 ,loss nan
epoch 26 ,loss nan
epoch 27 ,loss nan
epoch 28 ,loss nan
epoch 29 ,loss nan
epoch 30 ,loss nan
epoch 31 ,loss nan
epoch 32 ,loss nan
epoch 33 ,loss nan
epoch 34 ,loss nan
epoch 35 ,loss nan
epoch 36 ,loss nan
epoch 37 ,loss nan
epoch 38 ,loss nan
epoch 39 